In [ ]:
# utils.py

def transform_is_valid(transform, tolerance=1e-3):
    """
    checks if transformation is a valid SE(3)
    args:
        transform (numpy.array [4,4])
        tolerance (float, optional) 
    returns: 
        bool: True is `transform` is valid else False
    """
    shape = transform.shape == (4,4)
    isreal = np.all(np.isreal(transform))
    last_row = np.allclose(transform[3, :], [0, 0, 0, 1], atol=tolerance)
    R, T = transform[:3, :3], transform[:3, -1]
    RtR = R.transpose() @ R
    RRt = R @ R.transpose()
    det = np.linalg.det(R)
    valid = isreal and shape and np.allclose(RtR, np.eye(3), atol=tolerance) and np.allclose(RRt, np.eye(3), atol=tolerance) and np.isclose(det, 1, atol=tolerance)
    return valid


def camera_to_image(K, points):
    """
    project points from camera coordinate system to digital image plane
    args:
        K (numpy.array [3, 3])
        points [n, 3]
    returns:
        numpy.array [n, 2]: n 2D projections of the input points on the image plane
    """
    fx, fy = K[0,0], K[1,1] 
    cx, cy = K[0,2], K[1,2]
    x, y, z = points[:,0], points[:,1], points[:,2]
    u, v = (np.round((fx * (x/z) + cx))).astype(int), (np.round((fy * (y/z) + cy))).astype(int)
    points = np.vstack((u, v)).T
    return points


def world_to_camera(camera_pose, world_points):
    """
    Transform the points in the world frame to the camera frame, using camera pose.
    args:
        camera_pose (numpy.array [4, 4])
        points [n, 3]
    returns:
        numpy.array [n, 3]
    """
    world_coords_h = np.hstack((world_points, np.ones((world_points.shape[0], 1), dtype=np.float32)))
    camera_pose = np.linalg.inv(camera_pose)
    camera_coords_h = (camera_pose @ world_coords_h.T).T
    camera_coords_c = camera_coords_h[:, :3]
    return camera_coords_c


def depth_to_point_cloud(K, depth_image):
    """
    back project a depth image to a point cloud
    args:
        K (numpy.array [3, 3])
        depth_image (numpy.array [h, w]): each entry is a z depth value
    Returns:
        numpy.array [n, 3]: each row represents a different valid 3D point
    """
    fx, fy = K[0,0], K[1,1] 
    cx, cy = K[0,2], K[1,2]
    h, w = depth_image.shape
    u, v = np.meshgrid(np.arange(w), np.arange(h))
    u, v = u.flatten(u), u.flatten(v)
    Z = depth_image.flatten()
    valid = Z > 0
    u, v, Z = u[valid], v[valid], Z[valid]
    X, Y = (u - cx) * Z/fx, (v - cy) * Z/fy
    points = np.vstack((X, Y, Z)).T
    return points    


# test_rgbd_image_to_points(...) that will read an RGB-D image and convert it into point cloud and save it as test.ply.



.copy(order='C'): Ensures the array is stored in C-contiguous order, which is important for performance in certain NumPy operations.
astype(int): Converts the voxel bounds into integer values because the number of voxels must be a whole number.

Use **stack** when you want to add a new dimension.
Use **concatenate** when you want to merge along an existing dimension.

In [ ]:
# tsdf.py
import os
import cv2
import time
import numpy as np
from skimage import measure

class TSDFVolume:
    def __init__(self, volume_bounds, voxel_size):
        """
        args:
            volume_bounds (numpy.array [3, 2]): rows index [x, y, z] and cols index [min_bound, max_bound]. Units are in meters.
            voxel_size (float): The side length of each voxel in meters.
        """
        # raise value error if not conformant
        if volume_bounds.shape != (3,2):
            raise ValueError("Incorrect Shape")
        if voxel_size <= 0.0:
            raise ValueError("Invalid Size")
            
        """Numpy array [3, 2] of float32s, where rows index [x, y, z] and cols index [min_bound, max_bound]. Units are in meters."""
        self._volume_bounds = volume_bounds
        
        """float side length in meters of each 3D voxel cube."""
        self._voxel_size = float(voxel_size)
        
        """float tsdf truncation margin, the max allowable distance away from a surface in meters."""
        self._truncation_margin = 2 * self._voxel_size
        # adjust volume bounds
        self.voxel_bounds = np.ceil((self._volume_bounds[:,1]-self._volume_bounds[:,0])/self._voxel_size).copy(order='C').astype(int) 
        self._volume_bounds[:, 1] = self._volume_bounds[:, 0] + (self.voxel_bounds * self._voxel_size)
        
        """Origin of the voxel grid in world coordinate. Units are in meters."""
        self._volume_origin = self._volume_bounds[:,0].copy(order='C').astype(np.float32)
        print(f'Voxel volume size: {self.voxel_bounds[0]} x {self.voxel_bounds[1]} x {self.voxel_bounds[2]} - # voxels: {self.voxel_bounds[0] * self.voxel_bounds[1] * self.voxel_bounds[2]}')
        
        """Numpy array of float32s representing tsdf volume where each voxel represents a volume self._voxel_size^3. Shape of this volume is determined by (max_bound - min_bound)/ self._voxel_size. Each entry contains the distance to the nearest surface in meters, truncated by self._truncation_margin."""
        self._tsdf_volume = np.ones(self.voxel_bounds).astype(np.float32)
        
        """Numpy array of float32s with shape [self._tsdf_volume.shape, 3] in range [0.0, 255.0]. So each entry in the volume contains the average r, g, b color."""
        self._color_volume = np.zeros(np.append(self.voxel_bounds, 3)).astype(np.float32)
        
        """Numpy array [number of voxels, 3] of uint8s. [[0, 0, 0], [0, 0, 1], ...,]. When a new observation is made, we need to determine which of these voxel coordinates is "valid" so we can decide what voxels to update."""
        xv, yv, zv = np.meshgrid(
            range(self.voxel_bounds[0]),
            range(self.voxel_bounds[1]),
            range(self.voxel_bounds[2]),
        )
        self._voxel_coords = np.vstack((xv.flatten(), yv.flatten(), zv.flatten())).astype(int).T

    def get_mesh(self):
        """
        Reconstructs a 3D mesh from a tsdf volume using the marching cubes algorithm
        Returns:
            numpy.array [n, 3]: each row represents a 3D point.
            numpy.array [k, 3]: each row is a list of point indices used to render triangles.
            numpy.array [n, 3]: each row represents the normal vector for the corresponding 3D point.
            numpy.array [n, 3]: each row represents the color of the corresponding 3D point.
        """
        # Marching cubes
        voxel_points, triangles, normals, _ = measure.marching_cubes(self._tsdf_volume, level=0, method='lewiner')
        points_ind = np.round(voxel_points).astype(int)
        points = self.voxel_to_world(self._volume_origin, voxel_points, self._voxel_size)
        
        # Get vertex colors
        rgb_vals = self._color_volume[points_ind[:, 0], points_ind[:, 1], points_ind[:, 2]]
        colors_r, colors_g, colors_b = rgb_vals[:, 0], rgb_vals[:, 1], rgb_vals[:, 2]
        colors = np.floor(np.asarray([colors_r, colors_g, colors_b])).T
        colors = colors.astype(np.uint8)
        return points, triangles, normals, colors

    def get_valid_points(self, depth_image, voxel_u, voxel_v, voxel_z):
        """Compute a boolean array for indexing the voxel volume. Note that every time the method integrate(...) is called, not every voxel in the volume will be updated. This method returns a boolean matrix called valid_points with dimension (n, ), where n = # of voxels. Index i of valid_points will be true if this voxel will be updated, false if the voxel needs not to be updated."""
        image_height, image_width = depth_image.shape
        # Ensure voxel_u and voxel_v are valid integer indices
        voxel_u = voxel_u.astype(int)
        voxel_v = voxel_v.astype(int)
    
        # Eliminate pixels not in the image bounds or that are behind the image plane
        valid_pixels = np.logical_and(voxel_u >= 0,
                                      np.logical_and(voxel_u < image_width,
                                      np.logical_and(voxel_v >= 0,
                                      np.logical_and(voxel_v < image_height, voxel_z > 0))))
    
        # Get depths for valid coordinates u, v from the depth image. Zero elsewhere.
        depths = np.zeros(voxel_u.shape)
        depths[valid_pixels] = depth_image[voxel_v[valid_pixels], voxel_u[valid_pixels]]
    
        # Filter out zero depth values and depth + truncation margin >= voxel_z
        valid_points = np.logical_and(depths > 0, depths+self._truncation_margin >= voxel_z)
        return valid_points
        
    
    @staticmethod
    def voxel_to_world(volume_origin, voxel_size, voxel_coords):
        """Convert from voxel coordinates to world coordinates"""
        return volume_origin + voxel_coords * voxel_size
        
    @staticmethod
    def compute_tsdf(depth_image, voxel_z, truncation_margin, valid_points, valid_pixels):
        """Compute the new TSDF value for each valid point. We apply truncation and normalization in the end, so that tsdf value is in the range [-1,1]."""
        Z = voxel_z[valid_points]
        proj = np.zeros_like(Z)
        
        for idx, pixel in enumerate(valid_pixels):
            u, v = pixel[0], pixel[1]
            depth = depth_image[v, u]
            proj[idx] = depth - Z[idx]
            
        for idx, dist in enumerate(proj):
            proj[idx] = max(-1, min(1, dist/truncation_margin))

        tsdf = proj
        return tsdf

    @staticmethod
    def update_tsdf(tsdf_old, tsdf_new, color_old, color_new):
        """
        Update the TSDF value and color for the voxels that have new observations. 
        We only update the tsdf and color value when the new absolute value of tsdf_new[i] is smaller than that of tsdf_old[i].
        """
        for idx in range(len(tsdf_new)):
            if abs(tsdf_new[idx]) < abs(tsdf_old[idx]):
                tsdf_old[idx] = tsdf_new[idx]
                color_old[idx, :] = color_new[idx, :]
        return tsdf_old, color_old

    def integrate(self, color_image, depth_image, camera_intrinsics, camera_pose):
        """
        Integrate an RGB-D observation into the TSDF volume, by updating the tsdf volume, and color volume.
        Args:
            color_image (numpy.array [h, w, 3]): rgb image.
            depth_image (numpy.array [h, w]): 'z' depth image.
            camera_intrinsics (numpy.array [3, 3]): given as [[fu, 0, u0], [0, fv, v0], [0, 0, 1]]
            camera_pose (numpy.array [4, 4]): SE3 transform representing pose (camera to world)
        """
        color_image = color_image.astype(np.float32)
        depth_image = depth_image.astype(np.float32)
        voxel_world_coords = self.voxel_to_world(self._volume_origin, self._voxel_size, self._voxel_coords)
        voxel_camera_coords = world_to_camera(camera_pose, voxel_world_coords)
        voxel_img_coords = camera_to_image(camera_intrinsics, voxel_camera_coords)
        voxel_u, voxel_v, voxel_z = voxel_img_coords[:,0], voxel_img_coords[:,1], voxel_camera_coords[:,2]
        valid_points = self.get_valid_points(depth_image, voxel_u, voxel_v, voxel_z)
        valid_voxels = self._voxel_coords[valid_points]
        valid_pixels = voxel_img_coords[valid_points]
        tsdf = self.compute_tsdf(depth_image, voxel_z, self._truncation_margin, valid_points, valid_pixels)

        tsdf_old  = self._tsdf_volume[valid_voxels[:,0], valid_voxels[:,1], valid_voxels[:,2]]
        color_old = self._color_volume[valid_voxels[:,0], valid_voxels[:,1], valid_voxels[:,2]]
        color_new = color_image[valid_pixels[:,1].astype(int), valid_pixels[:,0].astype(int)]
        
        tsdf_updated, color_updated = self.update_tsdf(tsdf_old, tsdf, color_old, color_new)
        self._tsdf_volume[valid_voxels[:,0], valid_voxels[:,1], valid_voxels[:,2]] = tsdf_updated
        self._color_volume[valid_voxels[:,0], valid_voxels[:,1], valid_voxels[:,2]] = color_updated


if __name__ == "__main__":
    image_count = 10
    voxel_size = 0.01
    volume_bounds = np.array([[-0.75, 0.75], [-0.75,0.75], [0.,0.8]])
    camera_intrinsics = np.loadtxt("./data/camera-intrinsics.txt")
    tsdf_volume = TSDFVolume(volume_bounds, voxel_size)
    
    start_time = time.time()
    for i in range(image_count):
        print(f"Fusing frame {i+1}/{image_count}")
    
        # Load RGB-D image and camera pose
        color_image = cv2.imread(f"./data/frame-{i:06d}.color.png")
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        depth_image = cv2.imread(f"./data/frame-{i:06d}.depth.png", -1).astype(float) / 1000.0
        camera_pose = np.loadtxt(f"./data/frame-{i:06d}.pose.txt")
    
        # Integrate observation into voxel volume
        tsdf_volume.integrate(color_image, depth_image, camera_intrinsics, camera_pose)

    fps = image_count / (time.time() - start_time)
    print("Average FPS: {:.2f}".format(fps))

    # Get mesh from voxel volume and save to disk (can be viewed with Meshlab)
    print("Saving mesh to mesh.ply...")
    points, faces, normals, colors = tsdf_volume.get_mesh()
    mesh = Ply(triangles=faces, points=points, normals=normals, colors=colors)
    mesh.write('mesh_output.ply')
    print("Done")

In [ ]:
!pip install scikit-image

In [ ]:
import numpy as np
z = np.array([1,2,2,0,1])
filters = z > 0
z[filters]

In [ ]:
z.shape

In [ ]:
z.T.shape

In [ ]:
x = np.array([[1,3], 
              [3,4],
              [5,6]])
x[:,0].T.shape

In [ ]:
world_coords_h = np.hstack((world_points, np.ones(len(world_points), 1)), np.float32())
camera_coords_h = camera_pose @ homo_coords
camera_coords_c = camera_coords_h[:3]

In [ ]:
worlds_points = np.array([[1,2,3], [1,2,3]])
world_coords_h = np.hstack([worlds_points, np.ones((len(worlds_points), 1), np.float32)])
world_coords_h

In [ ]:
camera_pose = np.random.rand(4,4)
camera_coords_h = camera_pose @ world_coords_h.T
camera_coords_h

In [ ]:
np.loadtxt("./data/camera-intrinsics.txt")

In [ ]:
import os
os.listdir()

In [ ]:
!ls data/

In [ ]:
!pwd

In [ ]:
count = 0
for filename in os.listdir('./data'):
    if filename.endswith(".png"):
        count += 1
count

In [ ]:
import cv2
img_color = cv2.imread('frame-000000.color.png',cv2.IMREAD_COLOR)
img_color.shape

In [ ]:
!pip install opencv-python

In [ ]:
import numpy as np
import os

class Ply(object):
    """Class to represent a ply in memory, read plys, and write plys.
    """
    def __init__(self, ply_path=None, triangles=None, points=None, normals=None, colors=None):
        """Initialize the in memory ply representation.

        Args:
            ply_path (str, optional): Path to .ply file to read (note only
                supports text mode, not binary mode). Defaults to None.
            triangles (numpy.array [k, 3], optional): each row is a list of point indices used to
                render triangles. Defaults to None.
            points (numpy.array [n, 3], optional): each row represents a 3D point. Defaults to None.
            normals (numpy.array [n, 3], optional): each row represents the normal vector for the
                corresponding 3D point.. Defaults to None.
            colors (numpy.array [n, 3], optional): each row represents the color of the
                corresponding 3D point.. Defaults to None.
        """
        super().__init__()

        # If ply path is None, load in triangles, point, normals, colors.
        # else load ply from file. If ply_path is specified AND other inputs
        # are specified as well, ignore other inputs.
        # If normals are not None make sure that there are equal number of points and normals.
        # If colors are not None make sure that there are equal number of colors and normals.

        self.triangles = []
        self.points = []
        self.normals = []
        self.colors = []

        if ply_path is not None:
            assert os.path.exists(ply_path)
            self.read(ply_path)
        else:
            # Set from input args.
            if triangles is not None:
                self.triangles = triangles

            if points is not None:
                self.points = points

            if normals is not None:
                self.normals = normals

            if colors is not None:
                self.colors = colors

        if len(self.normals) != 0:
            assert len(self.normals) == len(self.points)

        if len(self.colors) != 0:
            assert len(self.colors) == len(self.points)


    def write(self, ply_path):
        """Write mesh, point cloud, or oriented point cloud to ply file.

        Args:
            ply_path (str): Output ply path.
        """
        with open(ply_path, 'w') as f:
            # Write header.
            f.write('ply\n')
            f.write('format ascii 1.0\n')
            f.write('element vertex {}\n'.format(len(self.points)))
            f.write('property float x\n')
            f.write('property float y\n')
            f.write('property float z\n')

            if len(self.normals) != 0:
                f.write('property float nx\n')
                f.write('property float ny\n')
                f.write('property float nz\n')

            if len(self.colors) != 0:
                f.write('property uchar red\n')
                f.write('property uchar green\n')
                f.write('property uchar blue\n')

            # Write faces header if dumping triangles.
            if len(self.triangles) != 0:
                f.write('element face {}\n'.format(len(self.triangles)))
                f.write('property list uchar int vertex_index\n')

            f.write('end_header\n')

            # Write points.
            for i in range(len(self.points)):
                f.write('{0} {1} {2}'.format(
                    self.points[i][0],
                    self.points[i][1],
                    self.points[i][2]))

                if len(self.normals) != 0:
                    f.write(' {0} {1} {2}'.format(
                        self.normals[i][0],
                        self.normals[i][1],
                        self.normals[i][2]))

                if len(self.colors) != 0:
                    f.write(' {0} {1} {2}'.format(
                        int(self.colors[i][0]),
                        int(self.colors[i][1]),
                        int(self.colors[i][2])))

                f.write('\n')

            # write triangles if they exist
            for triangle in self.triangles:
                f.write('3 {0} {1} {2}\n'.format(triangle[0], triangle[1], triangle[2]))

    def read(self, ply_path):
        """Read a ply into memory.

        Args:
            ply_path (str): ply to read in.
        """
        vertex_mode = False
        face_mode = False
        num_points = 0
        num_faces = 0
        index = 0

        self.points = []
        self.normals = []
        self.colors = []
        self.triangles = []

        parse_order = []

        with open(ply_path, 'r') as ps:
            for line in ps:
                line = line.split()

                if vertex_mode:
                    # Read in points and normals.
                    property_dict = {}

                    assert len(parse_order) == len(line)

                    for i, key in enumerate(parse_order):
                        property_dict[key] = float(line[i])

                    if ('x' in property_dict) and ('y' in property_dict) and ('z' in property_dict):
                        self.points.append([property_dict['x'], property_dict['y'], property_dict['z']])
                    if ('nx' in property_dict) and ('ny' in property_dict) and ('nz' in property_dict):
                        self.normals.append([property_dict['nx'], property_dict['ny'], property_dict['nz']])
                    if ('red' in property_dict) and ('green' in property_dict) and ('blue' in property_dict):
                        self.colors.append([property_dict['red'], property_dict['green'], property_dict['blue']])
                    index += 1
                    if index == num_points:
                        vertex_mode = False
                        face_mode = True
                        index = 0
                elif face_mode:
                    # Read in triangles.
                    self.triangles.append([int(i) for i in line[1:4]])
                    index += 1
                    if index == num_faces:
                        face_mode = False
                elif line[0] == 'element':
                    # set number of lines for vertices and faces.
                    if line[1] == 'vertex':
                        num_points = int(line[2])
                    elif line[1] == 'face':
                        num_faces = int(line[2])
                elif line[0] == 'property' and num_faces <= 0:
                    parse_order.append(line[2])
                elif line[0] == 'end_header':
                    vertex_mode = True


In [ ]:
# model.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class MiniUnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoders = nn.ModuleList([
            nn.Sequential(nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1), nn.ReLU()),
            nn.Sequential(nn.MaxPool2d(kernel_size=2), nn.Conv2d(16, 32, 3, padding=1), nn.ReLU()),
            nn.Sequential(nn.MaxPool2d(2), nn.Conv2d(32, 64, 3, padding=1), nn.ReLU()),
            nn.Sequential(nn.MaxPool2d(2), nn.Conv2d(64, 128, 3, padding=1), nn.ReLU()),
            nn.Sequential(nn.MaxPool2d(2), nn.Conv2d(128, 256, 3, padding=1), nn.ReLU()),
        ])
        self.decoders = nn.ModuleList([
            nn.Sequential(nn.Conv2d(128+256, 128, 3, padding=1)),
            nn.Sequential(nn.Conv2d(64+128, 64, 3, padding=1)),
            nn.Sequential(nn.Conv2d(32+64, 32, 3, padding=1)),
            nn.Sequential(nn.Conv2d(16+32, 16, 3, padding=1))
        ])
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        self.conv1x1 = nn.Conv2d(16, 6, 1)

    
    def forward(self, x):
        skips = []
        for encoder in self.encoders[:-1]:
            x = encoder(x)
            skips.append(x)
        x = self.encoders[-1](x)
        for idx, decoder in enumerate(self.decoders):
            x = self.upsample(x)
            x = torch.cat([x, skips[-(idx+1)]], dim=1)
            x = F.relu(decoder(x))
        return self.conv1x1(x)

In [ ]:
model = MiniUnet()
x = torch.rand(1,3,240,320)
output = model(x)
output.shape

In [ ]:
model

In [ ]:
params = list(model.parameters())
print(len(params))
print(params[0].shape)

In [ ]:
# datasete.py
import os
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms


class RGBDataset(Dataset):
    def __init__(self, root_dir, ground_truth=False, transform=True):
        self.root_dir = root_dir
        self.ground_truth = ground_truth
        self.transform = transform
        filenames = os.listdir(os.path.join(self.root_dir, 'rgb'))
        self.dataset_len = len(filenames)
    
    def __len__(self):
        return self.dataset_len

    def __getitem__(self, idx):
        rgb_img = cv2.imread(os.path.join(self.root_dir, 'rgb', f"{idx}_rgb.png"))
        rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
        
        if self.ground_truth:
            label = cv2.imread(os.path.join(self.root_dir, 'gt', f"{idx}_gt.png"))
            sample = {'image':rgb_img, 'label':label}
        else:
            sample = {'image':rgb_img}

        if self.transform:
            mean_rgb = [0.722, 0.751, 0.807]
            std_rgb = [0.171, 0.179, 0.197]
            transform = transforms.Compose([transforms.ToTensor(),
                                            transforms.Normalize(mean=mean_rgb, std=std_rgb),])
            sample['image'] = transform(sample['image'])
        return sample

    def display_sample(self, idx):
        sample = self.__getitem__(idx)
        plt.imshow(sample['image'].permute(1,2,0))
        plt.show()

In [ ]:
dataset = RGBDataset('/Users/rawri/Documents/GitHub/pose-estimation/dataset/train', True)

In [ ]:
!pwd

In [ ]:
dataset[2]['image'].shape

In [ ]:
dataset.display_sample(2)

In [ ]:
def main():
    select appropriate device
    read dataset
    transform dataset
    create dataloaders 
    instantitate model
    trainning loop
        forward pass
        loss : track train loss if using training data
        compute gradients thru backward pass
        perform optimization



In [ ]:
# utils.py

def save_checkpoint(model, epoch, val_miou, path='checkpoint.pth.tar'):
    """
    Save the model's state dictionary.
    """
    torch.save({
        'model_state_dict': model.state_dict(),
        'epoch': epoch,
        'model_miou': val_miou,
    }, path)
    print(f"Checkpoint saved at epoch {epoch}")


def load_checkpoint(model, path='checkpoint.pth.tar'):
    """
    Load a saved model checkpoint.
    """
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model, checkpoint['epoch'], checkpoint['model_miou']


def plot_learning_curve(train_loss_list, train_miou_list, val_loss_list, val_miou_list, path='learning_curve.png'):
    """
    Plot and save the training and validation loss/mIoU curves.
    """
    epochs = np.arange(1, len(train_loss_list) + 1)
    plt.figure()
    plt.plot(epochs, train_loss_list, label="train_loss", color='navy')
    plt.plot(epochs, train_miou_list, label="train_mIoU", color='teal')
    plt.plot(epochs, val_loss_list, label="val_loss", color='orange')
    plt.plot(epochs, val_miou_list, label="val_mIoU", color='gold')
    plt.legend(loc='upper left')
    plt.xlabel('Epoch')
    plt.ylabel('mIoU / Loss')
    plt.grid(True)
    plt.savefig(path, bbox_inches='tight')
    plt.show()


def save_predictions(model, device, dataloader, dataset_dir):
    """
    Save model predictions to the specified directory.
    """
    pred_dir = Path(dataset_dir) / 'pred'
    pred_dir.mkdir(parents=True, exist_ok=True)
    print(f"Saving predicted masks to {pred_dir}")

    model.to(device).eval()
    with torch.no_grad():
        for batch_id, batch in enumerate(dataloader):
            inputs = batch['input'].to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, dim=1)

            for i in range(preds.size(0)):
                scene_id = batch_id * dataloader.batch_size + i
                mask = preds[i].cpu().numpy()
                mask_path = pred_dir / f"{scene_id}_pred.png"
                show_mask(mask)
                image.write_mask(mask, str(mask_path))


def iou(prediction, target):
    """
    Compute IoU for each class excluding the background.
    """
    _, pred = torch.max(prediction, dim=1)
    batch_size = prediction.size(0)
    class_num = prediction.size(1)
    batch_ious = []

    for batch_id in range(batch_size):
        class_ious = []
        for class_id in range(1, class_num):  # Skip background class
            mask_pred = (pred[batch_id] == class_id).int()
            mask_target = (target[batch_id] == class_id).int()
            if mask_target.sum() == 0:  # Skip if target is not present
                continue
            intersection = (mask_pred * mask_target).sum().item()
            union = mask_pred.sum().item() + mask_target.sum().item() - intersection
            class_ious.append(float(intersection) / float(union) if union > 0 else 0.0)
        batch_ious.append(np.mean(class_ious) if class_ious else 0.0)
    
    return batch_ious



In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader

import image
from dataset import RGBDataset
from model import MiniUNet
from segmentation_helper import check_dataset, check_dataloader, show_mask

def run_epoch(model, device, dataloader, criterion, optimizer=None):
    """
    Run a single epoch of training or validation.
    """
    is_training = optimizer is not None
    model.train() if is_training else model.eval()

    epoch_loss, epoch_iou, data_size = 0, 0, 0

    for batch in dataloader:
        inputs = batch['input'].to(device)
        targets = batch['target'].to(device)
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        batch_size = inputs.size(0)

        epoch_loss += loss.item() * batch_size
        epoch_iou += np.sum(iou(outputs, targets))
        data_size += batch_size

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return epoch_loss / data_size, epoch_iou / data_size


def main():
    # Check for device (GPU/CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Directories setup
    root_dir = Path('./dataset/')
    train_dir = root_dir / 'train'
    val_dir = root_dir / 'val'
    test_dir = root_dir / 'test'

    # Load datasets and dataloaders
    BATCH_SIZE = 4
    NUM_WORKERS = 2
    train_dataset = RGBDataset(train_dir, has_gt=True)
    val_dataset = RGBDataset(val_dir, has_gt=True)
    test_dataset = RGBDataset(test_dir, has_gt=False)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    # Initialize model, loss, and optimizer
    model = MiniUNet().to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training and validation loop
    num_epochs = 15
    best_miou = float('-inf')
    history = {'train_loss': [], 'train_miou': [], 'val_loss': [], 'val_miou': []}

    for epoch in range(1, num_epochs + 1):
        print(f'Epoch {epoch}/{num_epochs}')

        train_loss, train_miou = run_epoch(model, device, train_loader, criterion, optimizer)
        val_loss, val_miou = run_epoch(model, device, val_loader, criterion)

        history['train_loss'].append(train_loss)
        history['train_miou'].append(train_miou)
        history['val_loss'].append(val_loss)
        history['val_miou'].append(val_miou)

        print(f"Train loss/mIoU: {train_loss:.2f}/{train_miou:.2f}")
        print(f"Validation loss/mIoU: {val_loss:.2f}/{val_miou:.2f}")

        if val_miou > best_miou:
            best_miou = val_miou
            save_checkpoint(model, epoch, val_miou)

    # Load the best model and make predictions
    model, best_epoch, best_miou = load_checkpoint(model)
    save_predictions(model, device, val_loader, val_dir)
    save_predictions(model, device, test_loader, test_dir)

    # Plot and save learning curves
    plot_learning_curve(history['train_loss'], history['train_miou'], history['val_loss'], history['val_miou'])


if __name__ == '__main__':
    main()

In [1]:
import torch